In [1]:
import pandas as pd
import yfinance as yf

#Read the CSV file containing ticker symbols
bnks_data = pd.read_csv('resources/BNKS_holdings.csv')
bnks_data.dropna(inplace=True)

# Extracting ticker symbols into a list
ticker_list = bnks_data['Ticker'].tolist()

In [2]:
# Define a function to fetch cash flow data for a given ticker symbol
def fetch_cash_flow(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        cash_flow = ticker.cashflow  # Fetching cash flow data
        return cash_flow
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")
        return None

# Create a dictionary to store cash flow data for each ticker
cash_flows = {}

# Iterating over each ticker symbol and fetching its cash flow data
for ticker in ticker_list:
    cash_flow_data = fetch_cash_flow(ticker)
    if cash_flow_data is not None:
        cash_flows[ticker] = cash_flow_data

In [3]:
# Ticker symbols for which you want to view the cash flow data
ticker_symbols = ['BAC', 'JPM', 'WFC']

# Accessing and printing the cash flow data for each specified ticker
for ticker_symbol in ticker_symbols:
    banks_cash_flow = cash_flows.get(ticker_symbol)

    if banks_cash_flow is not None:
        print(f"\nCash Flow Statement for {ticker_symbol}:")
        print(banks_cash_flow)
    else:
        print(f"Cash flow data for {ticker_symbol} is not available.")


Cash Flow Statement for BAC:
                                                    2022-12-31  \
Free Cash Flow                                   -6327000000.0   
Repurchase Of Capital Stock                      -5727000000.0   
Repayment Of Debt                               -34055000000.0   
Issuance Of Debt                                 65910000000.0   
Issuance Of Capital Stock                         4426000000.0   
Interest Paid Supplemental Data                  18526000000.0   
Income Tax Paid Supplemental Data                 2288000000.0   
End Cash Position                               230203000000.0   
Beginning Cash Position                         348221000000.0   
Effect Of Exchange Rate Changes                  -3123000000.0   
Changes In Cash                                -114895000000.0   
Financing Cash Flow                            -106039000000.0   
Cash Flow From Continuing Financing Activities -106039000000.0   
Net Other Financing Charges                   